# **Final Project Report**
## **1. Introduction**
An ongoing research project by UBC's Pacific Laboratory for Artificial Intelligence (PLAI) is researching the gaming behaviours of Minecraft players to improve the behaviour of characters that players interact with (called non-playable characters). Their team has provided data for the basis of this analysis. To ensure that enough data is available for the project, candidates who contribute a large amount of data must be selected to participate in the research group's trials. Hence, we aim to investigate the following question: 
    
"What demographic of players in the dataset are most likely to contribute a large amount of data?"

To specify, we will be focusing on the experience level, age, and gender of the players as the determining factors of the demographic. We realize that the appeal of a game varies based on experience, so knowing the experience level of players who play the most helps recruitment efforts. Players of different ages also respond differently to visuals and promotional language used in recruitment, allowing ads to target by age. Participants who are likely to contribute large amounts of data will be based on the amount of time spent gaming, where more hours spent playing equates to more data.

### **Dataset Description**
Frank Wood's research group provided all available data by monitoring their Minecraft server and recording in-game actions. We will be using the players.csv dataset for this analysis. It includes 196 total observations and 9 variables, of which age and played_hours are quantitative variables, while the rest are categorical variables. Figure 1.1 below describes the contents of the dataset:

| Variable| Description |
| --- | --- |
| experience | level of gaming experience (Amateur, Beginner, Regular, Pro, Veteran)|
| subscribe | subscription status (True or False)|
| hashedEmail | encrypted email |
| played_hours |time played (in hours)|
| name |name of player|
| gender | gender of players (Male, Female, Non-binary, Two-spirited, etc.)|
| age | age of players (in years) |
| individualId | ID of player |
| organizationName | organization of player |

Figure 1.1: Variables and Descriptions
    
## **2. Method and Results**
We began by reading the dataset and wrangling the data into a tidy format before the analysis. Although there are 9 total variables in the dataset, the hashedEmail and individualId variables include no observations, so they were omitted from our analysis during the preliminary processing. 

Next, summary statistics such as the mean, median, and standard deviations for the dataset were determined, and simple visualizations were created to gain further insight into the data. 

Finally, a KNN-regression analysis was performed with age and the level of experience as the predictor variables and played_hours as the response variable. As the KNN model is based on Euclidean distances, it could not be applied directly to the level of experience since the variable is categorical. Instead, a dictionary with 'dummy variables' was created and assigned to the different levels of experience to represent each value numerically (i.e. 1 for beginner, 5 for veteran). The dataset is split into 75% training and 25% testing subsets to perform the KNN-regression analysis, where the KNN-regression model was trained using cross-validation with the training subset to determine the optimal k-value and then evaluated with the testing data. Predictions resulting from the model were plotted and analyzed to determine an ultimate conclusion about the demographic of players that are predicted to spend the most time playing in response to our initial research question.

### **Preliminary Data Processing**
#### **1. Import Relevant Packages**

In [119]:
# Importing all packages needed for data processing, visualization, and following analysis

import altair as alt
import numpy as np
import pandas as pd

#### **2. Load Dataset**

In [120]:
# Loading read_csv dataset for processing.

player_data = pd.read_csv("playersdata/players.csv")
player_data

,experience,subscribe,hashedEmail,played_hours,name,gender,age,individualId,organizationName
0,Pro,True,f6daba428a5e19a3d47574858c13550499be23603422e6...,30.3,Morgan,Male,9,NaN,NaN
1,Veteran,True,f3c813577c458ba0dfef80996f8f32c93b6e8af1fa9397...,3.8,Christian,Male,17,NaN,NaN
2,Veteran,False,b674dd7ee0d24096d1c019615ce4d12b20fcbff12d79d3...,0.0,Blake,Male,17,NaN,NaN
3,Amateur,True,23fe711e0e3b77f1da7aa221ab1192afe21648d47d2b4f...,0.7,Flora,Female,21,NaN,NaN
4,Regular,True,7dc01f10bf20671ecfccdac23812b1b415acd42c2147cb...,0.1,Kylie,Male,21,NaN,NaN
...,...,...,...,...,...,...,...,...,...
191,Amateur,True,b6e9e593b9ec51c5e335457341c324c34a2239531e1890...,0.0,Bailey,Female,17,NaN,NaN
192,Veteran,False,71453e425f07d10da4fa2b349c83e73ccdf0fb3312f778...,0.3,Pascal,Male,22,NaN,NaN
193,Amateur,False,d572f391d452b76ea2d7e5e53a3d38bfd7499c7399db29...,0.0,Dylan,Prefer not to say,17,NaN,NaN
194,Amateur,False,f19e136ddde68f365afc860c725ccff54307dedd13968e...,2.3,Harlow,Male,17,NaN,NaN


Figure 2.1: Raw Players Data

#### **3. Cleaning and Wrangling**

In [121]:
# Only keep relevant variables by dropping columns that contain no observations.

player_tidy= player_data.drop(columns=['individualId', 'organizationName'])
player_tidy

,experience,subscribe,hashedEmail,played_hours,name,gender,age
0,Pro,True,f6daba428a5e19a3d47574858c13550499be23603422e6...,30.3,Morgan,Male,9
1,Veteran,True,f3c813577c458ba0dfef80996f8f32c93b6e8af1fa9397...,3.8,Christian,Male,17
2,Veteran,False,b674dd7ee0d24096d1c019615ce4d12b20fcbff12d79d3...,0.0,Blake,Male,17
3,Amateur,True,23fe711e0e3b77f1da7aa221ab1192afe21648d47d2b4f...,0.7,Flora,Female,21
4,Regular,True,7dc01f10bf20671ecfccdac23812b1b415acd42c2147cb...,0.1,Kylie,Male,21
...,...,...,...,...,...,...,...
191,Amateur,True,b6e9e593b9ec51c5e335457341c324c34a2239531e1890...,0.0,Bailey,Female,17
192,Veteran,False,71453e425f07d10da4fa2b349c83e73ccdf0fb3312f778...,0.3,Pascal,Male,22
193,Amateur,False,d572f391d452b76ea2d7e5e53a3d38bfd7499c7399db29...,0.0,Dylan,Prefer not to say,17
194,Amateur,False,f19e136ddde68f365afc860c725ccff54307dedd13968e...,2.3,Harlow,Male,17


Figure 2.2: Tidied Players Data

#### **4. Summary Statistics**

In [122]:
# Determining the mean, median, and standard deviations for the played hours for each variable: experience, gender, and age.

experience_aggregated = player_tidy.groupby("experience")["played_hours"].agg(
    ["mean", "median", "std"]
).reset_index()

experience_aggregated.rename(columns = {
    "mean" : "playedhours_mean", "median" : "playedhours_median", "std" : "playedhours_std"
})

,experience,playedhours_mean,playedhours_median,playedhours_std
0,Amateur,6.017460,0.1,21.987214
1,Beginner,1.248571,0.1,4.105547
2,Pro,2.600000,0.3,7.988838
3,Regular,18.208333,0.1,57.957346
4,Veteran,0.647917,0.1,1.937424


Figure 2.3: Experience Summary Values

In [123]:
gender_aggregated = player_tidy.groupby("gender")["played_hours"].agg(
    ["mean", "median", "std"]
).reset_index()

gender_aggregated.rename(columns = {
    "mean" : "playedhours_mean", "median" : "playedhours_median", "std" : "playedhours_std"
})

,gender,playedhours_mean,playedhours_median,playedhours_std
0,Agender,6.250000,6.25,8.838835
1,Female,10.635135,0.10,38.167301
2,Male,4.127419,0.10,21.531838
3,Non-binary,14.880000,0.00,56.224133
4,Other,0.200000,0.20,NaN
5,Prefer not to say,0.372727,0.00,0.681309
6,Two-Spirited,0.083333,0.00,0.160208


Figure 2.4: Experience Summary Values

In [124]:
age_aggregated = player_tidy.groupby("age")["played_hours"].agg(
    ["mean", "median", "std"]
).reset_index()

age_aggregated.rename(columns = {
    "mean" : "playedhours_mean", "median" : "playedhours_median", "std" : "playedhours_std"
})

,age,playedhours_mean,playedhours_median,playedhours_std
0,8,0.300000,0.30,NaN
1,9,30.300000,30.30,NaN
2,10,3.600000,3.60,NaN
3,11,2.900000,2.90,NaN
4,12,0.500000,0.50,NaN
5,14,8.700000,8.70,12.020815
6,15,0.100000,0.10,0.000000
7,16,50.033333,0.10,86.573687
8,17,4.757333,0.00,26.872347
9,18,0.228571,0.10,0.325137


Figure 2.3: Age Summary Values

Simple observations of the mean and variance show that values in experience and gender variables are within a reasonable range while possible outliers exist for the age variable. Purely based on the summary statistics, we would expect regular players within the range of 14 to 20 years old to be the greatest contributors of data due to the highest mean played hours. However, this is only derived from the exploratory analysis to fain greater understanding of the data and does not take into account the number of observations in each category, demonstrating the need for further analysis in order to form concrete conclusions.

### **Preliminary Data Visualization**
#### **1. Visualizing Numerical Variables**

In order to explore the possible relationship between the age and played_hours variables, a scatterplot was used to visualize the distribution of the data points. The scatterplot shows no distinct trends or relationships, although it seems that most of the participants are between the ages of 15-25 years old and played for less that 20 hours each. This was determined from how most of the datapoints are clustered along the bottom of the graph.

In [125]:
age_plot= alt.Chart(player_tidy, title = "Age vs. Hours Played").mark_point(opacity=0.4).encode(
    x= alt.X('age', axis=alt.Axis(tickCount=5)).title('Age'),
    y= alt.Y('played_hours').title('Time Played (hrs)'),
    color=alt.Color('played_hours', scale=alt.Scale(scheme='category20b')
).title("Hours Played")).properties(
        width = 300,
        height = 300
)
                                                                                   
age_plot                                                                       

alt.Chart(...)

Figure 2.4: Age vs. Time Played plot

#### **2. Visualizing Categorical Variables**

Bar graphs were created for Experience vs. Hours Played and Gender vs. Hours Played to show any major variations among categories that could help provide a basis for our final prediction and analysis. Observation of the Experience vs. Hours Played plot suggests that players with regular gaming experience had a greater average number of hours played. The Gender vs. Hours Played plot shows that participants who are non-binary had the most hours played on average. However, this may not be representative as the mean data was graphed instead of raw data to prevent major gaps in data. Doing so only determines the average and does not account for the number of participants in each category.

In [126]:
experience_aggregated = player_tidy.groupby("experience")["played_hours"].mean().reset_index()

experience_plot= alt.Chart(experience_aggregated, title = "Experience Level vs. Hours Played").mark_bar().encode(
    x= alt.X('experience').title('Experience Level'),
    y= alt.Y('played_hours').title('Time Played (hrs)'),
    color=alt.Color('experience', legend=alt.Legend(title='Experience'), scale=alt.Scale(scheme='lightmulti'))
).properties(
        width = 200,
        height = 300
).configure_axis(
    labelAngle=-45
)
                                                                                   
experience_plot   

alt.Chart(...)

Figure 2.5: Experience vs. Time Played plot

In [127]:
gender_aggregated = player_tidy.groupby("gender")["played_hours"].mean().reset_index()

gender_plot= alt.Chart(gender_aggregated, title = "Gender vs. Hours Played").mark_bar().encode(
    x= alt.X('gender').title('Gender'),
    y= alt.Y('played_hours').title('Time Played (hrs)'),
    color=alt.Color('gender', legend=alt.Legend(title='Gender'), scale=alt.Scale(scheme='lightmulti')
)).properties(
        width = 200,
        height = 300
).configure_axis(
    labelAngle=-45
)
                                                                                   
gender_plot

alt.Chart(...)

Figure 2.6: Gender vs. Time Played plot

In [128]:
# Visualizing the gender distribution across experience levels using raw data

experience_gendered_plot= alt.Chart(player_tidy, title = "Experience and Gender vs. Time Played").mark_bar().encode(
    x= alt.X('experience').title('Player Experience'),
    y= alt.Y('played_hours').title('Time Played (hrs)'),
    color=alt.Color('gender', legend=alt.Legend(title='Gender'), scale=alt.Scale(scheme='lightmulti')
)).properties(
        width = 200,
        height = 300
).configure_axis(
    labelAngle=-45
)

experience_gendered_plot

alt.Chart(...)

Figure 2.7: Experience and Gender vs. Time Played Plot

### **Analysis Model**

In [129]:
# # Importing additional packages needed for performing a KNN Regression

import matplotlib.pyplot as plt 
import altair as alt
import pandas as pd
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline
from sklearn import set_config
from sklearn.preprocessing import OneHotEncoder, StandardScaler

In [130]:
player_tidy = player_data.iloc[:, [0,3,6]]

new = {
    'Veteran':5,
    'Pro':4,
    'Regular':3,
    'Amateur':2,
    'Beginner':1
}

player_tidy['experience'] = player_tidy['experience'].replace(new)

player_tidy

/tmp/ipykernel_4351/4019014021.py:11: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  player_tidy['experience'] = player_tidy['experience'].replace(new)
/tmp/ipykernel_4351/4019014021.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  player_tidy['experience'] = player_tidy['experience'].replace(new)


,experience,played_hours,age
0,4,30.3,9
1,5,3.8,17
2,5,0.0,17
3,2,0.7,21
4,3,0.1,21
...,...,...,...
191,2,0.0,17
192,5,0.3,22
193,2,0.0,17
194,2,2.3,17


In [131]:
# Split the data sets into training and testing data

player_train, player_test = train_test_split(
    player_tidy, train_size=0.75,
    random_state = 2000     # setting the random state to ensure the results are reproducible
)

In [132]:
# import the K-NN regression model
from sklearn.neighbors import KNeighborsRegressor

# preprocess the data, make the pipeline
preprocessor = make_column_transformer((StandardScaler(), ['age', 'experience']))
pipeline = make_pipeline(preprocessor, KNeighborsRegressor())

# create the 5-fold GridSearchCV object
param_grid = {
    "kneighborsregressor__n_neighbors": range(1, 50),
}
gridsearch = GridSearchCV(
    estimator=preprocessor,
    param_grid=param_grid,
    cv=5,
    scoring="neg_root_mean_squared_error",
)

pipeline

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('standardscaler',
                                                  StandardScaler(),
                                                  ['age', 'experience'])])),
                ('kneighborsregressor', KNeighborsRegressor())])

In [133]:
# Define the preprocessing step
preprocessor = make_column_transformer(
    (StandardScaler(), ['age', 'experience'])
)

# Create the pipeline
pipeline = make_pipeline(
    preprocessor,  # Preprocessing step
    KNeighborsRegressor()  # KNeighborsRegressor as the model
)

# Define the parameter grid for K tuning
param_grid = {
    'kneighborsregressor__n_neighbors': range(1,70)  # Values to test for n_neighbors
}

# Initialize GridSearchCV
gridsearch = GridSearchCV(
    estimator=pipeline,
    param_grid=param_grid,
    cv=5,  # 5-fold cross-validation
    scoring='neg_mean_squared_error'
)

# Fit the GridSearchCV object
gridsearch.fit(
    player_train[['age', 'experience']],  # Features
    player_train['played_hours']  # Outcome variable
)

# Retrieve cross-validation results
player_results = pd.DataFrame(gridsearch.cv_results_)
player_results['sem_test_score'] = player_results['std_test_score'] / 5**(1/2)
player_results = (
    player_results[[
        'param_kneighborsregressor__n_neighbors',  # Column for n_neighbors
        'mean_test_score',  # Mean test score
        'sem_test_score'  # Standard error of the mean
    ]]
    .rename(columns={'param_kneighborsregressor__n_neighbors': 'n_neighbors'})  # Rename for clarity
)

player_results

,n_neighbors,mean_test_score,sem_test_score
0,1,-1156.345575,420.459611
1,2,-828.928863,308.259928
2,3,-738.960184,308.886884
3,4,-767.195575,296.049072
4,5,-766.543560,289.243768
...,...,...,...
64,65,-614.933875,300.931187
65,66,-615.913282,300.819785
66,67,-615.608487,301.210033
67,68,-615.880439,301.470993


In [134]:
player_results["mean_test_score"] = player_results["mean_test_score"]
player_results

,n_neighbors,mean_test_score,sem_test_score
0,1,-1156.345575,420.459611
1,2,-828.928863,308.259928
2,3,-738.960184,308.886884
3,4,-767.195575,296.049072
4,5,-766.543560,289.243768
...,...,...,...
64,65,-614.933875,300.931187
65,66,-615.913282,300.819785
66,67,-615.608487,301.210033
67,68,-615.880439,301.470993


In [135]:
# Find the optimal K value (representing the number of nearest neighbours) for the data set

gridsearch.best_params_

{'kneighborsregressor__n_neighbors': 34}

In [136]:
from sklearn.metrics import mean_squared_error

# Create a new column with the predicted values on the testing data set

player_test["predicted"] = gridsearch.predict(player_test)

player_test

,experience,played_hours,age,predicted
111,3,4.0,21,13.129412
73,5,0.0,22,0.905882
3,2,0.7,21,4.352941
149,2,0.0,16,3.297059
80,5,0.0,17,0.547059
172,5,0.0,20,0.573529
61,3,1.5,20,12.191176
157,3,0.1,99,3.652941
65,5,0.1,21,0.941176
50,5,0.6,21,0.941176


In [137]:
# Calculate the root mean square prediction error for the test set

RMSPE = mean_squared_error(
    y_true=player_test["played_hours"],
    y_pred=player_test["predicted"]
)**(1/2)

RMSPE

np.float64(36.72268606138095)

In [138]:
# Create a grid of evenly spaced values for both 'age' and 'experience'
age_prediction_grid = pd.DataFrame({
    "age": player_tidy['age'],
    "experience": player_tidy['experience']
})

# Predict played hours for each combination in the grid
age_prediction_grid["predicted"] = gridsearch.predict(age_prediction_grid)

# Base plot: scatter plot of actual player data
base_plot = alt.Chart(player_tidy).mark_circle(opacity=0.4).encode(
    x=alt.X("age", scale=alt.Scale(zero=False), title="Age"),
    y=alt.Y("played_hours", scale=alt.Scale(domain=[1, 250]), axis=alt.Axis(format=",.0f"), title="Hours Played"),
    color=alt.Color('experience:N')
)

# Add the predictions as a line
predictions_plot = base_plot + alt.Chart(age_prediction_grid).mark_line(color="red").encode(
    x="age",
    y="predicted"
)

# Display the final plot
predictions_plot

alt.LayerChart(...)

### **Discussion**

#### **1. Visualization Analysis**

Considering the bar graph between experience level and time played, players with a 'Regular' experience level logged the most playing hours. This graph was based on mean played hours, showing that this experience level's playing time was a mean of over 18 hours; this was a significantly higher mean than the amateur level, who logged a mean of 6 played hours. Additionally, Figure 2.6 shows the graph of the mean played hours for players of different genders. The visualization revealed that players who are non-binary and female have the highest mean time played, logging about 14.8 and 10.7 hours, respectively. These two visualizations were combined in Figure 2.7, which used the raw data to show the distribution of genders within each experience level. The bar graph shows the same trend, in which individuals with regular experience log the most hours, followed by amateurs. Within the regular experience level, it was mainly composed of female, male, and non-binary players fairly evenly distributed. Similarly, the amateur players mostly identified as male or female. These visualizations communicate that the people with the most hours are logged by regular-experience players who are male, female, and non-binary.

#### **2. KNN Regression Analysis**